In [118]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [119]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPool2D, Flatten
from keras import optimizers
from keras import losses
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random # for visualization
print('Libraries Imported')

In [184]:
#imgs_path = '../input/mias-mammography/all-mias/'
imgs_path='../input/ganpat/Processed/'

In [185]:
from PIL import Image
import glob
x= []
for filename in sorted(glob.glob("../input/ganpat/Processed/*.png")): 
    img=cv2.imread(filename)
    #print(img.shape)
    img =cv2.resize(img,(224, 224))
    x.append(img)

In [186]:
x[0].shape

In [187]:
print(x[0])

In [188]:
plt.imshow(x[0])

In [189]:
fig=plt.figure(figsize=(10,15))
columns = 3
rows = 3
for i in range(1, columns*rows +1):
    img = np.random.randint(10)
    fig.add_subplot(rows, columns, i)
    plt.imshow(x[i])
plt.show()

In [190]:
header_list = ["REFNUM", "BG", "CLASS", "SEVERITY", "X", "Y", " RADIUS"]
#index	REFNUM	BG	CLASS	SEVERITY	X	Y	RADIUS

In [191]:
path='../input/minor-project-dataset/Processed MIAS/MIAS Annotations.csv'
info=pd.read_csv(path,names=header_list)

In [192]:
info.head()

In [193]:
list(info)

In [194]:
info['SEVERITY'].fillna(0)

In [195]:
info.shape

In [196]:
info.isnull().sum()

In [197]:
info.dtypes

In [198]:
info['SEVERITY'].value_counts()

In [199]:
info['BG'].value_counts()

In [200]:
info['CLASS'].value_counts()

In [201]:
ids = {}
for i in range(len(info)):
    ids[i] = info.REFNUM[i]

In [202]:
# Turning our outputs Normal-Abnormal to 0-1
label = []
for i in range(len(info)):
    if info.CLASS[i] != 'NORM':
        label.append(1)
    else:
        label.append(0)
label = np.array(label)
print(f"Imagenes: {len(label)}, Normales = {len(label)-np.sum(label)}, No normales= {np.sum(label)}")

In [203]:
# define the every images filepaths in to list
img_name = []
for i in range(len(label)):
        img_name.append(imgs_path+info.REFNUM[i]+'.png')

In [204]:
count = 0
remove = True
temp_label = []
temp_img_name = []

for i, lbl in enumerate(label.tolist()):
    if lbl == 0 and remove == True:
        count = count + 1
        if count >= 84:
            remove = False
    else:
        temp_label.append(lbl)
        temp_img_name.append(img_name[i])

In [205]:
label = np.array(temp_label)
img_name = temp_img_name

In [206]:
img_name = np.array(img_name)

In [207]:
#print(img_name)
print(f'image addres amount {img_name.shape}')

In [208]:
def view_25_random_image():
    fig = plt.figure(figsize = (15, 10))
    for i in range(25):
        rand = random.randint(0,len(label))
        print(img_name[rand],0)

random_images = view_25_random_image()

In [209]:
def view_25_random_image():
    fig = plt.figure(figsize = (15, 10))
    for i in range(25):
        rand = random.randint(0,len(label))
        ax = plt.subplot(5, 5, i+1)
        img = cv2.imread(img_name[rand], 0)
        #plt.imshow(img)
        img = cv2.resize(img, (256,256))
        if label[rand] == 1:
            plt.title('Not Notmal')
        else:
            plt.title('Normal')
        plt.tight_layout()
        plt.axis('off')
        plt.imshow(img)
    fig.savefig('random_25_image_fig.png')

random_images = view_25_random_image()

In [210]:
img_path = []
last_label = []
IMG_SIZE = 224
Angle=[15,30,45,60,75,90,105,120,135,150,-15,-30,-45,-60,-75,-90,-105,-120,-135,-150]
for i in range(len(img_name)):
    
    img = cv2.imread(img_name[i], 0)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    rows, cols= img.shape
    for angle in Angle:
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle*2, 1)    #Rotate 0 degree
            img_rotated = cv2.warpAffine(img, M, (IMG_SIZE, IMG_SIZE))
            img_path.append(img_rotated)
            if label[i] == 1:
                last_label.append(1)
            else:
                last_label.append(0)

In [211]:
img_path = np.array(img_path)
last_label = np.array(last_label)
img_path.shape

In [212]:
img_path = img_path.reshape((-1,IMG_SIZE,IMG_SIZE,1))
img_path.shape

In [213]:
last_label.shape

In [214]:
# split train and test set
x_train, x_test, y_train, y_test = train_test_split(img_path, last_label, test_size = 0.2, random_state = 42)

In [215]:
len(x_train),len(x_test),len(y_train),len(y_test)

In [216]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output
from tqdm.notebook import tqdm
import cv2
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.activations import *
from keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from time import time

In [217]:
reg = tf.keras.regularizers.L2(l2=1e-5)
ini = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02, seed=None)


model = Sequential()
model.add(Conv2D(32,
                 kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer = ini,
                 input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(Conv2D(64,
                 kernel_size=(3,3),
                 kernel_initializer = ini,
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3,3),
                 kernel_initializer = ini,
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Dense(64, 
                kernel_initializer = ini,
                activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-3)

model.compile(optimizer=optimizer,
              loss= 'binary_crossentropy',
              metrics=['accuracy'])

In [218]:
model.summary()

In [219]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=5 ,restore_best_weights=True, verbose=1)

check_point_filepath = './'

model_check_point = ModelCheckpoint(filepath =check_point_filepath, monitor='val_loss', verbose=1, save_best_only=True,
                                    save_weights_only=False, mode='auto', save_freq='epoch')

In [220]:
train = True
if train:
    epochs=5
    history = model.fit(x_train,
                 y_train,
                 validation_split=0.2,
                 epochs=epochs,
                 batch_size=128,
                 callbacks=[early_stop, model_check_point])
else:
    model = tf.keras.models.load_model('./')

In [221]:
from random import randint
test_num = randint(0,len(x_test))
#test_num = 274
result = model.predict(np.array([x_test[test_num]]))

plt.figure(figsize= (10,10))
plt.imshow(x_test[test_num])

print(f"The results was: {result[0][0]>= 0.5}       And the probability being: {result[0][0]}\nReal value was: {y_test[test_num]>=0.5}")
print(f"Test id: {test_num}")

In [222]:
history.history.keys()

In [223]:
plt.figure(figsize=(10,5))
plt.plot(range(len(history.history['accuracy'])),history.history['accuracy'], label= "accuracy")
plt.plot(range(0,len(history.history['val_accuracy'])),history.history['val_accuracy'], label= "val_accuracy")
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Epoch")
plt.legend()
plt.show()

In [224]:
plt.figure(figsize=(10,5))
plt.plot(range(0,len(history.history['loss'])),history.history['loss'], label= "loss")
plt.plot(range(0,len(history.history['val_loss'])),history.history['val_loss'], label= "val_loss")
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs Epoch")
plt.legend()
plt.show()

In [225]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)